## This notebook adds geospatial data from one dataframe to another. It uses fuzzy matching between military installation names 

### all of the data was scraped from a Department of Defense's  Per- and polyfluoroalkyl substances (PFAS)  website: https://www.acq.osd.mil/eie/eer/ecc/pfas/map/pfasmap.html

In [1]:
import os
import pandas as pd
import plotly.express as px
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt
from fuzzywuzzy import process
import numpy as np

os.chdir(r'C:\Users\OITNYNWilsoS\OneDrive - Department of Veterans Affairs\python\PACT Act - MET\PDF data\geo')
cwd = os.getcwd()
print("Current working directory is:", cwd)

Current working directory is: C:\Users\OITNYNWilsoS\OneDrive - Department of Veterans Affairs\python\PACT Act - MET\PDF data\geo


In [2]:
#metadata_df is a stored table of 703 military installations and their spatial data. This data was webscraped
# from DOD's PFAS website. See the "DAPM-PFAS-web-scrape-and-map" repository on Veterans Affairs GitHub account. 

%store -r
metadata_df

geo = metadata_df

#make the installation names uppercase to standardize
geo['InstallationName'] = geo['InstallationName'].str.upper()
#rename column header
geo = geo.rename(columns={'InstallationName': 'SITE_NAME'})
geo.head()

,Id,DodComponentId,DodComponent,State,SITE_NAME,InstallationType,Latitude,Longitude
0,1,2100,Army,Virgin Islands,AAOF BLAIR HANGAR,National Guard,17.725062,-64.782945
1,2,2100,Army,Alabama,AASF #1 R W SHEPHERD HOPE HULL,National Guard,32.287216,-86.394073
2,3,2100,Army,Alabama,AASF #2 BIRMINGHAM,National Guard,33.572333,-86.750542
3,4,2100,Army,Tennessee,AASF #2 TN,National Guard,35.821234,-83.994949
4,5,2100,Army,Washington,AASF #2 WA,National Guard,47.618166,-117.641352


In [3]:
#optional export of spatial data to .csv
#geo.to_csv(r'C:\Users\OITNYNWilsoS\OneDrive - Department of Veterans Affairs\GIS\MET DOD website PFAS scraping\lat_long_PFAS_DOD_website.csv', float_format='{:f}'.format, encoding='utf-8', index=False)

In [17]:
#pull in the data from March 2018 publically avilable DOD report on PFAS in drinking water. This data was scraped from
#a PDF file. See "DAPM-PFAS-PDF-scrape" repoisitory on VA's GitHub account. 

%store -r
df_2018.head()

,branch,name,state,date,number systems tested,number above EPA LHA,Results (PFOS/ PFOA),"Actions Taken as of August 31, 2017",Actions Planned to reduce,purveyor,on or off base,results,analyte,name_geo,similarity
0,USMC,MCB CAMP PENDLETON SOUTH,California,2017,1,1,77,One sample exceeded the PFOS/PFOA combined Hea...,Installation will continue to monitor the system.,DOD,on,77.0,NaN,CAMP PENDLETON CA MCB,87
1,Army,99TH RSC MARTINSBURG MEMORIAL USARC,West Virginia,2017,1,1,0-79/0-71,The results reported were the range of PFOS/PF...,NaN,non-DOD,on,79.0,NaN,99TH RSC MARTINSBURG MEMORIAL USARC (MARTINSBURG),95
2,Army,EL CAMPO,Texas,2017,1,1,79,Well is designated as non-potable. Bottled wat...,Additional sampling will be conducted in Novem...,DOD,on,79.0,NaN,EL CAMPO,100
3,Army,"SOTO CANO AB, HN",Honduras,2017,1,1,PFOA + PFOS combined = 72.5- 82.9,"Initial, confirmatory and subsequent sampling ...",NaN,DOD,on,82.9,PFOA+PFOS,"SOTO CANO AB, HN",100
4,Air Force - ACTIVE,NEW BOSTON AFS,New Hampshire,2017,2,1,13-83 ppt (PFOS+PFOA),Drinking water sampling and analysis. DW well ...,Project has been awarded to add Activated Carb...,DOD,on,83.0,PFOA+PFOS,NEW BOSTON AFS,100


In [5]:
#code to determine number of unique installations
unique = df_2018["name"].nunique()
unique

55

In [6]:
#rename military bases from notebook that compare mapped and 2018 data
df_2018['name'] = df_2018['name'].str.upper()
df_2018['name'] = df_2018['name'] .apply(lambda x: x.replace('(','').replace(')','')) 
df_2018['name'] = df_2018['name'].str.replace('CAMP GRAYLING JOINT MANEUVER TRAINING CENTER','CAMP GRAYLING MTC')
df_2018['name'] = df_2018['name'].str.replace('NAS WHITING FIELD MAIN BASE','WHITING FLD FL NAS')
df_2018['name'] = df_2018['name'].str.replace('HORSHAM AGS AGS 111TH','BIDDLE ANGB')
df_2018['name'] = df_2018['name'].str.replace('NAS WHIDBEY ISLAND - AULT FIELD PRIVATE','WHIDBEY IS WA NAS')
df_2018['name'] = df_2018['name'].str.replace('NAS WHIDBEY ISLAND - OLF COUPEVILLE PRIVATEE','WHIDBEY IS WA NAS')
df_2018['name'] = df_2018['name'].str.replace('NAS OCEANA - NALF FENTRESS','OCEANA VA NAS')
df_2018['name'] = df_2018['name'].str.replace('PEASE ANGB 157TH','PEASE')
df_2018.head()

,branch,name,state,date,number systems tested,number above EPA LHA,Results (PFOS/ PFOA),"Actions Taken as of August 31, 2017",Actions Planned to reduce,purveyor,on or off base,results,analyte
0,USMC,MCB CAMP PENDLETON SOUTH,California,2017,1,1,77,One sample exceeded the PFOS/PFOA combined Hea...,Installation will continue to monitor the system.,DOD,on,77.0,NaN
1,Army,99TH RSC MARTINSBURG MEMORIAL USARC,West Virginia,2017,1,1,0-79/0-71,The results reported were the range of PFOS/PF...,NaN,non-DOD,on,79.0,NaN
2,Army,EL CAMPO,Texas,2017,1,1,79,Well is designated as non-potable. Bottled wat...,Additional sampling will be conducted in Novem...,DOD,on,79.0,NaN
3,Army,"SOTO CANO AB, HN",Honduras,2017,1,1,PFOA + PFOS combined = 72.5- 82.9,"Initial, confirmatory and subsequent sampling ...",NaN,DOD,on,82.9,PFOA+PFOS
4,Air Force - ACTIVE,NEW BOSTON AFS,New Hampshire,2017,2,1,13-83 ppt (PFOS+PFOA),Drinking water sampling and analysis. DW well ...,Project has been awarded to add Activated Carb...,DOD,on,83.0,PFOA+PFOS


In [7]:
#Inital fuzzy matching on the spatial and 2018 data only provided partial success. Since I wanted to capture as many
#installations as possible, I edited the names in the spatial dataframe to match the names in the 2018 dataframe. 
#This allowed me to apply spatial data to 48 out of 55 unique installations listed in the 2018 dataframe,
#though a very manual process.

geo['SITE_NAME'] = geo['SITE_NAME'].str.replace('EWVRA SHEPHERD FIELD', '99TH RSC MARTINSBURG MEMORIAL USARC') #, regex=True)
geo['SITE_NAME'] = geo['SITE_NAME'].str.replace('SOTO CANO', 'SOTO CANO AB, HN')
geo['SITE_NAME'] = geo['SITE_NAME'].str.replace('NEW BOSTON AIR FORCE STATION', 'NEW BOSTON AFS')
geo['SITE_NAME'] = geo['SITE_NAME'].str.replace('KUNSAN AB', 'KUNSAN AB')
geo['SITE_NAME'] = geo['SITE_NAME'].str.replace('HARRISBURG INTERNATIONAL AIRPORT', 'HARRISBURG IAP ANG 193RD')
geo['SITE_NAME'] = geo['SITE_NAME'].str.replace('SEAL BEACH CA WPNSUPFAC', 'WEAPONS STATION SEAL BEACH- NOSC MORENO')
geo['SITE_NAME'] = geo['SITE_NAME'].str.replace('BARNES MUNICIPAL', 'BARNES ANGB 104TH')
geo['SITE_NAME'] = geo['SITE_NAME'].str.replace('STEWART INTERNATIONAL AIRPORT','MARINE FORCES RESERVE MARFORRES-MARINE CORPS RESERVE TRAINING CENTER MCRTC STEWART')
geo['SITE_NAME'] = geo['SITE_NAME'].str.replace('BARROW AK NARL', 'NAVAL BASE KITSAP - NARL BARROW')
geo['SITE_NAME'] = geo['SITE_NAME'].str.replace('FRANCIS S. GABRESKI', 'GABRESKI ANGB 106TH', regex=True)
geo['SITE_NAME'] = geo['SITE_NAME'].str.replace('WHIDBEY IS WA NAS', 'NAS WHIDBEY ISLAND')
geo['SITE_NAME'] = geo['SITE_NAME'].str.replace('CAMP WALKER', 'USAG DAEGU, KR CAMP WALKER')
geo['SITE_NAME'] = geo['SITE_NAME'].str.replace('NAVSUPPACT Monterey CA', 'NSA MONTEREY - NAVAL RADIO TRANSMITTER FACILITY DIXON')
geo['SITE_NAME'] = geo['SITE_NAME'].str.replace('FORMER PEASE AFB', 'PEASE AFB')
geo['SITE_NAME'] = geo['SITE_NAME'].str.replace('WILLOW GROVE NASJRB', 'WILLOW GROVE')
geo['SITE_NAME'] = geo['SITE_NAME'].str.replace('CAMP CARROLL', 'USAG DAEGU, KR CAMP CARROLL')
geo['SITE_NAME'] = geo['SITE_NAME'].str.replace('WARMINSTER NAWC AD', 'WARMINSTER PUBLIC')
geo['SITE_NAME'] = geo['SITE_NAME'].str.replace('WILMINGTON ARMORY', 'MARFORRES-MCRTC WILMINGTON')
geo['SITE_NAME'] = geo['SITE_NAME'].str.replace('PETERSON SPACE FORCE BASE', 'PETERSON AFB')
geo['SITE_NAME'] = geo['SITE_NAME'].str.replace('DOVER AFB', 'DOVER AFB')
geo.head()

,Id,DodComponentId,DodComponent,State,SITE_NAME,InstallationType,Latitude,Longitude
0,1,2100,Army,Virgin Islands,AAOF BLAIR HANGAR,National Guard,17.725062,-64.782945
1,2,2100,Army,Alabama,AASF #1 R W SHEPHERD HOPE HULL,National Guard,32.287216,-86.394073
2,3,2100,Army,Alabama,AASF #2 BIRMINGHAM,National Guard,33.572333,-86.750542
3,4,2100,Army,Tennessee,AASF #2 TN,National Guard,35.821234,-83.994949
4,5,2100,Army,Washington,AASF #2 WA,National Guard,47.618166,-117.641352


In [8]:
#use fuzzy matching compare names in the spatial and 2018 dataframe. Create a column called "similiarity"
#that reports the score of the most similar names between the two. 
name_geo = []
similarity = []
for i in df_2018.name:
        ratio = process.extract(i, geo.SITE_NAME, limit=1)
        name_geo.append(ratio[0][0])
        similarity.append(ratio[0][1])
df_2018['name_geo'] = pd.Series(name_geo)
df_2018['name_geo'] = df_2018['name_geo']
df_2018['similarity'] = pd.Series(similarity)
df_2018 = df_2018.sort_values(by=['similarity'], ascending=True)
df_2018.head(n=8)

,branch,name,state,date,number systems tested,number above EPA LHA,Results (PFOS/ PFOA),"Actions Taken as of August 31, 2017",Actions Planned to reduce,purveyor,on or off base,results,analyte,name_geo,similarity
55,Navy,NSF DIEGO GARCIA,Diego Garcia,2017,5,3,"77-5,849",Alternate drinking water was already being pro...,Continued monitoring for PFOS and PFOA at the ...,DOD,on,5849.0,NaN,NAVBASE SAN DIEGO,58
44,Navy - BRAC,WARMINSTER PRIVATE,Pennsylvania,2017,345,65,"71 - 1,800",Bottled water provided. Connection to Public w...,Continue to connect to Public water. Approxima...,non-DOD,off,1800.0,NaN,WARMINSTER PUBLIC,74
52,Navy,WHIDBEY IS WA NAS,Washington,2017,103,2,"140 - 3,823",Addressed current exposure. Bottled water to t...,Various options currently being evaluated; res...,non-DOD,off,3823.0,NaN,AASF #2 WA,86
26,Navy,NSA MONTEREY - NAVAL RADIO TRANSMITTER FACILIT...,California,2017,1,1,260,Drinking water sampling and analysis. Navy is ...,Will continue to track issues as the state inv...,DOD,on,260.0,NaN,NAVAL BASE KITSAP - NARL BARROW,86
31,Army,"USAG RED CLOUD, KR: CAMP RED CLOUD",South Korea,2017,1,1,PFOA + PFOS combined = 171- 466,"Initial, confirmatory and subsequent sampling ...",Continue to use Army system wells producing wa...,DOD,on,466.0,PFOA+PFOS,CAMP LEJEUNE NC MCB,86
6,Army,81ST RSC: E. EARLE RIVES AFRC,North Carolina,2017,1,1,<40-90,PFOS per 2014 Consumer Confidence Report. The ...,NaN,non-DOD,on,90.0,NaN,99TH RSC MARTINSBURG MEMORIAL USARC (MARTINSBURG),86
8,Army,"USAG BENELUX - CASERNE DAUMERIE, BE",Belgium,2017,1,1,PFOA + PFOS combined = 84-94,Purchased water - Initial and confirmatory sam...,Continued use of bottled water until closure/t...,non-DOD,on,94.0,PFOA+PFOS,USAG ITALY - VICENZA,86
0,USMC,MCB CAMP PENDLETON SOUTH,California,2017,1,1,77,One sample exceeded the PFOS/PFOA combined Hea...,Installation will continue to monitor the system.,DOD,on,77.0,NaN,CAMP PENDLETON CA MCB,87


In [9]:
#only keep rows with a similarity match greater than or equal to 87.
#After visual inspection, anything less than 87 resulted in a poor match between installation names. 
df_2018_87 = df_2018[df_2018["similarity"] >= 87]
df_2018_87 = df_2018_87.reset_index(drop=True)
df_2018_87.head()

,branch,name,state,date,number systems tested,number above EPA LHA,Results (PFOS/ PFOA),"Actions Taken as of August 31, 2017",Actions Planned to reduce,purveyor,on or off base,results,analyte,name_geo,similarity
0,USMC,MCB CAMP PENDLETON SOUTH,California,2017,1,1,77,One sample exceeded the PFOS/PFOA combined Hea...,Installation will continue to monitor the system.,DOD,on,77.0,NaN,CAMP PENDLETON CA MCB,87
1,Navy - BRAC,AGANA,Guam,2017,12,5,88 - 410,Sampling conducted by Guam Waterworks Authorit...,NaN,non-DOD,off,410.0,NaN,GUAM AGANA NAS,90
2,Air Force - ACTIVE,JOINT BASE CAPE COD,Massachusetts,2017,9,2,73 - 488 ppt (PFOS +PFOA),"Sampling and analysis, retesting, two (2) publ...",Wellhead treatment on public DW water supply w...,non-DOD,off,488.0,PFOA+PFOS,OTIS ANG (JOINT BASE CAPE COD -MASSACHUSETTS M...,90
3,Army,FT. LEAVENWORTH,Kansas,2017,1,1,PFOA + PFOS combined = 97-649,American Water owns this system with the wells...,Contract award to study new source well - late...,non-DOD,on,649.0,PFOA+PFOS,FORT LEAVENWORTH,90
4,Navy,NAS WHIDBEY ISLAND - OLF COUPEVILLE PRIVATE,Washington,2017,100,7,130 - 660,Bottled water to 9 residents (2 of 7 wells ser...,Various options currently being evaluated; res...,non-DOD,off,660.0,NaN,NAS WHIDBEY ISLAND,90


In [10]:
#select and rename the columns from the spatial dataframe that need to be attached to the 2018 dataframe
geo_select = geo[['SITE_NAME','Latitude','Longitude']]
geo_select = geo_select.rename(columns = {'SITE_NAME': 'name_geo', 
                                'Latitude':'latitude',
                                'Longitude':'longitude'})

geo_select.head()

,name_geo,latitude,longitude
0,AAOF BLAIR HANGAR,17.725062,-64.782945
1,AASF #1 R W SHEPHERD HOPE HULL,32.287216,-86.394073
2,AASF #2 BIRMINGHAM,33.572333,-86.750542
3,AASF #2 TN,35.821234,-83.994949
4,AASF #2 WA,47.618166,-117.641352


In [11]:
#merged 2018 dataframe with similarity scores >=87 and the selected dataframe columns. This attaches the spatial data
#to the 2018 dataframe if there was a good name match. 
df_2018_87_geo = df_2018_87.merge(geo_select, on='name_geo', how='left')
df_2018_87_geo.head()

,branch,name,state,date,number systems tested,number above EPA LHA,Results (PFOS/ PFOA),"Actions Taken as of August 31, 2017",Actions Planned to reduce,purveyor,on or off base,results,analyte,name_geo,similarity,latitude,longitude
0,USMC,MCB CAMP PENDLETON SOUTH,California,2017,1,1,77,One sample exceeded the PFOS/PFOA combined Hea...,Installation will continue to monitor the system.,DOD,on,77.0,NaN,CAMP PENDLETON CA MCB,87,33.361871,-117.423732
1,Navy - BRAC,AGANA,Guam,2017,12,5,88 - 410,Sampling conducted by Guam Waterworks Authorit...,NaN,non-DOD,off,410.0,NaN,GUAM AGANA NAS,90,13.473339,144.824168
2,Air Force - ACTIVE,JOINT BASE CAPE COD,Massachusetts,2017,9,2,73 - 488 ppt (PFOS +PFOA),"Sampling and analysis, retesting, two (2) publ...",Wellhead treatment on public DW water supply w...,non-DOD,off,488.0,PFOA+PFOS,OTIS ANG (JOINT BASE CAPE COD -MASSACHUSETTS M...,90,41.658201,-70.521583
3,Army,FT. LEAVENWORTH,Kansas,2017,1,1,PFOA + PFOS combined = 97-649,American Water owns this system with the wells...,Contract award to study new source well - late...,non-DOD,on,649.0,PFOA+PFOS,FORT LEAVENWORTH,90,39.364170,-94.918251
4,Navy,NAS WHIDBEY ISLAND - OLF COUPEVILLE PRIVATE,Washington,2017,100,7,130 - 660,Bottled water to 9 residents (2 of 7 wells ser...,Various options currently being evaluated; res...,non-DOD,off,660.0,NaN,NAS WHIDBEY ISLAND,90,48.338638,-122.661501


In [12]:
df_2018_geo = df_2018_87_geo
#select only key columns for ArcGIS map
df_2018_geo = df_2018_geo[['state',
                            'branch',  
                            'name',
                            'date',
                            'results',
                            'purveyor',
                            'on or off base',
                            'analyte',
                            'longitude',
                            'latitude']]

df_2018_geo.head()

,state,branch,name,date,results,purveyor,on or off base,analyte,longitude,latitude
0,California,USMC,MCB CAMP PENDLETON SOUTH,2017,77.0,DOD,on,NaN,-117.423732,33.361871
1,Guam,Navy - BRAC,AGANA,2017,410.0,non-DOD,off,NaN,144.824168,13.473339
2,Massachusetts,Air Force - ACTIVE,JOINT BASE CAPE COD,2017,488.0,non-DOD,off,PFOA+PFOS,-70.521583,41.658201
3,Kansas,Army,FT. LEAVENWORTH,2017,649.0,non-DOD,on,PFOA+PFOS,-94.918251,39.364170
4,Washington,Navy,NAS WHIDBEY ISLAND - OLF COUPEVILLE PRIVATE,2017,660.0,non-DOD,off,NaN,-122.661501,48.338638


In [13]:
#calculate number of unique installations with spatial data
unique = df_2018_geo["name"].nunique()
unique

48

In [14]:
#export 2018 geocoded data to csv to use in ArcGIS
#df_2018_geo.to_csv(r'C:\Users\OITNYNWilsoS\OneDrive - Department of Veterans Affairs\GIS\MET DOD website PFAS scraping\geo_2018.csv', float_format='{:f}'.format, encoding='utf-8', index=False)

In [15]:
#df_2018_geo.to_csv(r"C:\Users\OITNYNWilsoS\OneDrive - Department of Veterans Affairs\PACT Act\MET\deliverables\geo_max_2018.csv", float_format='{:f}'.format, encoding='utf-8', index=False)